## Find Best models

In [6]:
from wandb.apis.public import Api
from datetime import datetime, timedelta
import pandas as pd

In [7]:
ARCHITECTURE_MAP = {
    "moat": {
        "smaller": 1,
        "default": 2,
        "bigger2": 4,
        "bigger": 3,
    },
    "coatnet": {
        "first": 1,
        "default": 2,
        "bigger2": 4,
        "bigger": 3,
    },
    "swintransformer": {
        "small": 1,
        "default": 2,
        "bigger": 4,
        "big": 3,

    },
}
DATASET_MAP = {
    "all": ["all", "practical", "mka", "noiseless"],
    "custom": ["custom", "custom_mac", "custom_open_window", "custom_dishwasher", "custom_washing_machine"],
    "custom_noisy": [ "custom_noisy", "custom_open_window", "custom_dishwasher", "custom_washing_machine"]
}

In [8]:
def get_all_runs():
    runs = Api().runs(
        path="przybytniowskaj-warsaw-university-of-technology/master-thesis",
        filters={}
    )

    runs_data = []
    for run in runs:
        dt = datetime.strptime(run.created_at, "%Y-%m-%dT%H:%M:%SZ")
        suffix = '' if not run.config.get("special_keys") else '_unmapped'
        model = run.config.get("model", "Unknown")
        if model == "swintransformer":
            last_month = datetime.now() - timedelta(days=30)
            if not (run.name.endswith(suffix) and dt > last_month):
                continue
        else:
            if not (run.name.endswith(suffix) and dt > datetime.fromtimestamp(1746067200)):
                continue

        architecture = run.config.get("model_params", "Unknown")
        if model not in ARCHITECTURE_MAP:
            continue

        mapped_arch = None
        for key, value in ARCHITECTURE_MAP[model].items():
            if key == architecture.split('_')[0]:
                mapped_arch = value
                if "window" in architecture:
                    mapped_arch = f"{value}_{architecture.split('_')[-1]}"
                break
        if mapped_arch is None:
            continue

        runs_data.append({
            "Name": run.name,
            "Model": model,
            "Architecture": mapped_arch,
            "Dataset": run.config.get("dataset", "Unknown"),
            "Special_Keys": run.config.get("special_keys", False),
            **{key: round(value * 100, 2) if isinstance(value, (int, float)) and "acc" in key.lower() else value
               for key, value in run.summary.items() if key.startswith("test_")}
        })

    return pd.DataFrame(runs_data)

In [9]:
def filter_for_dataset(runs, dataset, special_keys):
    runs = runs[runs['Dataset'] == dataset].copy()

    cols = ['Name', 'Model', 'Architecture', 'Special_Keys']
    suffixes = DATASET_MAP[dataset] if dataset in DATASET_MAP else [dataset]
    if dataset == "all":
        datasets = ["all"]
    else:
        datasets = ["custom_mac", "custom_noisy"]
    cols += [
        col for col in runs.columns
        if (
            (col.startswith('test_acc1_') and any(col.endswith(suffix) for suffix in suffixes))
            or any(col.endswith(dataset_x) for dataset_x in datasets)
            or (col.startswith('test_loss_') and any(col.endswith(suffix) for suffix in suffixes))
        )
        and not col.endswith("all_w_custom_noisy") and not col.endswith("all_w_custom")
    ]
    runs = runs[cols]
    if len(suffixes) > 1:
        runs[f'test_loss_{dataset}'] = round(runs[[col for col in runs.columns if col.startswith('test_loss_')]].mean(axis=1), 4)
    else:
        runs[f'test_loss_{suffixes[0]}'] = round(runs[f'test_loss_{suffixes[0]}'], 4)
    runs.drop(columns=[col for col in runs.columns if col.startswith('test_loss_') and col != f'test_loss_{dataset}'], inplace=True)

    runs = runs.loc[runs.groupby(["Model", "Architecture", "Special_Keys"])[f"test_acc1_{dataset}"].idxmax()]
    runs = runs[runs['Special_Keys'] == special_keys].copy()

    runs.drop(columns=['Special_Keys'], inplace=True)
    runs = runs.sort_values(by=f"test_acc1_{dataset}", ascending=False)

    return runs

In [10]:
runs = get_all_runs()
runs.head()

KeyboardInterrupt: 

In [ ]:
all_alphanum = filter_for_dataset(runs, "all", False)
all_alphanum.to_csv("../results_summary/all_alphanum.csv", index=False)
all_alphanum


,Name,Model,Architecture,test_acc10_all,test_acc1_all,test_acc1_mka,test_acc1_noiseless,test_acc1_practical,test_acc2_all,test_acc3_all,test_acc4_all,test_acc5_all,test_loss_all
65,coatnet_bigger_adam_CosineAnnealingLR_lr_0.000...,coatnet,3,99.53,92.21,83.89,97.06,95.00,96.57,97.82,98.29,98.75,0.0028
195,coatnet_bigger2_adam_CosineAnnealingLR_lr_0.00...,coatnet,4,99.38,90.65,85.00,96.08,91.94,97.20,98.13,98.75,98.91,0.0027
72,moat_default_window_adam_CosineAnnealingLR_lr_...,moat,2_window,99.69,88.16,79.44,92.16,91.39,95.48,97.04,97.66,98.75,0.0082
183,moat_bigger2_window8_adam_CosineAnnealingLR_lr...,moat,4_window8,99.38,86.60,80.00,91.18,88.61,94.70,97.20,97.98,98.29,0.0042
524,moat_smaller_window_adam_CosineAnnealingLR_lr_...,moat,1_window,99.07,86.29,82.78,91.18,86.67,93.93,97.04,97.66,98.29,0.0060
78,moat_bigger_adam_CosineAnnealingLR_lr_3e-05_wd...,moat,3,99.22,85.98,78.33,89.22,88.89,94.39,96.42,98.13,98.91,0.0085
208,moat_bigger2_window16_adam_CosineAnnealingLR_l...,moat,4_window16,99.53,85.67,77.22,89.22,88.89,94.86,96.73,97.51,98.13,0.0050
533,coatnet_first_adam_CosineAnnealingLR_lr_0.0001...,coatnet,1,99.07,84.27,71.67,92.16,88.33,92.37,95.48,97.04,98.44,0.0059
59,coatnet_default_adam_chained_lr_5e-05_partial_...,coatnet,2,99.53,83.64,75.00,88.24,86.67,92.52,94.70,96.73,97.82,0.0064
68,moat_default_adam_CosineAnnealingLR_lr_0.0001_...,moat,2,99.38,83.33,75.00,89.22,85.83,92.83,96.11,97.51,98.60,0.0056


In [ ]:
all_all = filter_for_dataset(runs, "all", True)
all_all.to_csv("../results_summary/all_all.csv", index=False)
all_all


,Name,Model,Architecture,test_acc10_all,test_acc1_all,test_acc1_mka,test_acc1_noiseless,test_acc1_practical,test_acc2_all,test_acc3_all,test_acc4_all,test_acc5_all,test_loss_all
85,coatnet_bigger_adam_CosineAnnealingLR_lr_0.000...,coatnet,3,99.48,90.94,85.08,98.31,93.33,97.02,98.19,98.58,98.84,0.0024
192,coatnet_bigger2_adam_chained_lr_0.0001_partial...,coatnet,4,99.35,90.82,87.12,93.22,93.06,96.64,97.80,98.58,98.97,0.0020
87,moat_bigger_adam_CosineAnnealingLR_lr_5e-05_wd...,moat,3,99.61,89.65,84.07,95.76,92.22,96.38,98.06,98.45,98.58,0.0034
513,moat_smaller_adam_CosineAnnealingLR_lr_0.0001_...,moat,1,99.35,87.71,85.42,88.14,89.44,94.70,97.41,98.32,98.84,0.0037
83,moat_bigger_window_adam_chained_lr_5e-05_wd_0....,moat,3_window,98.84,86.03,79.66,93.22,88.89,94.44,96.64,97.67,98.06,0.0078
517,moat_smaller_window_adam_CosineAnnealingLR_lr_...,moat,1_window,99.09,85.77,83.73,87.29,86.94,93.79,96.38,97.67,98.19,0.0043
84,moat_default_window_adam_CosineAnnealingLR_lr_...,moat,2_window,99.09,84.48,82.03,89.83,84.72,93.92,95.73,96.90,98.06,0.0044
531,coatnet_first_adam_chained_lr_0.0001_partial_w...,coatnet,1,99.35,83.31,79.32,89.83,84.44,92.76,95.86,97.41,98.19,0.0044
89,coatnet_default_adam_chained_lr_5e-05_partial_...,coatnet,2,98.84,78.78,76.27,79.66,80.56,91.33,93.66,95.73,97.28,0.0069
491,swintransformer_big_adam_chained_lr_0.0001_wd_...,swintransformer,3,93.79,58.47,53.56,73.73,57.50,74.26,80.47,84.48,87.32,0.0101


In [ ]:
custom_mac_alphanum = filter_for_dataset(runs, "custom_mac", False)
custom_mac_alphanum.to_csv("../results_summary/custom_mac_alphanum.csv", index=False)
custom_mac_alphanum

,Name,Model,Architecture,test_acc10_custom_mac,test_acc10_custom_noisy,test_acc1_custom_mac,test_acc1_custom_noisy,test_acc2_custom_mac,test_acc2_custom_noisy,test_acc3_custom_mac,test_acc3_custom_noisy,test_acc4_custom_mac,test_acc4_custom_noisy,test_acc5_custom_mac,test_acc5_custom_noisy,test_loss_custom_mac
327,moat_bigger_adam_CosineAnnealingLR_lr_0.0001_w...,moat,3,95.00,51.39,70.00,8.36,91.00,15.48,93.00,18.89,94.00,24.77,94.00,31.89,0.0577
333,moat_bigger_window_adam_CosineAnnealingLR_lr_0...,moat,3_window,95.00,46.13,65.00,8.98,84.00,14.24,89.00,19.50,94.00,24.46,94.00,29.10,0.0765
339,moat_bigger2_adam_chained_lr_0.0001_wd_0.05_sp...,moat,4,95.00,46.44,61.00,7.74,81.00,13.62,88.00,20.74,93.00,25.39,93.00,28.17,0.2620
343,moat_bigger2_window8_adam_chained_lr_0.0001_wd...,moat,4_window8,92.00,47.68,58.00,6.19,80.00,12.38,84.00,18.89,90.00,22.60,90.00,28.79,0.2965
134,moat_default_adam_chained_lr_0.0001_wd_0.05_sp...,moat,2,98.00,37.77,55.00,5.88,78.00,11.15,88.00,15.79,94.00,19.81,94.00,23.84,0.0403
344,moat_bigger2_window16_adam_CosineAnnealingLR_l...,moat,4_window16,93.00,44.27,52.00,6.19,71.00,13.00,83.00,17.96,89.00,23.22,89.00,27.86,0.3304
70,moat_default_window_adam_chained_lr_0.0001_wd_...,moat,2_window,90.74,45.51,51.85,5.88,77.78,12.38,84.26,16.10,87.96,21.36,87.96,26.32,0.1613
230,coatnet_bigger2_adam_chained_lr_0.0001_partial...,coatnet,4,81.00,39.32,50.00,4.33,66.00,10.53,75.00,14.86,79.00,18.27,79.00,24.46,0.0992
50,coatnet_default_adam_CosineAnnealingLR_lr_0.00...,coatnet,2,94.44,38.39,48.15,4.33,62.96,8.05,72.22,13.62,79.63,19.20,85.19,23.22,1.9454
234,coatnet_bigger_adam_chained_lr_0.0001_partial_...,coatnet,3,85.00,37.46,47.00,4.95,69.00,9.91,79.00,12.38,82.00,16.72,82.00,19.81,0.1005


In [ ]:
custom_mac_all = filter_for_dataset(runs, "custom_mac", True)
custom_mac_all.to_csv("../results_summary/custom_mac_all.csv", index=False)
custom_mac_all


,Name,Model,Architecture,test_acc10_custom_mac,test_acc10_custom_noisy,test_acc1_custom_mac,test_acc1_custom_noisy,test_acc2_custom_mac,test_acc2_custom_noisy,test_acc3_custom_mac,test_acc3_custom_noisy,test_acc4_custom_mac,test_acc4_custom_noisy,test_acc5_custom_mac,test_acc5_custom_noisy,test_loss_custom_mac
315,moat_bigger_window_adam_chained_lr_0.0001_wd_0...,moat,3_window,98.82,46.45,87.57,13.48,94.67,20.57,98.22,24.11,98.22,28.19,98.22,31.56,0.0149
214,coatnet_bigger2_adam_chained_lr_0.0001_partial...,coatnet,4,99.41,52.84,86.39,12.41,95.86,20.39,98.22,25.71,98.82,31.74,98.82,35.82,0.0124
311,moat_bigger_adam_chained_lr_0.0001_wd_0.05_spe...,moat,3,98.22,47.70,86.39,10.82,96.45,20.92,97.04,26.77,98.22,31.56,98.22,35.46,0.0155
222,coatnet_first_adam_chained_lr_0.0001_partial_w...,coatnet,1,100.00,43.97,81.66,9.04,96.45,15.60,98.22,22.34,100.00,26.24,100.00,31.91,0.0212
216,coatnet_bigger_adam_CosineAnnealingLR_lr_0.000...,coatnet,3,97.04,48.40,77.51,11.17,90.53,19.86,92.90,25.00,96.45,29.61,96.45,33.87,0.0284
319,moat_bigger2_adam_chained_lr_0.0001_wd_0.05_sp...,moat,4,95.86,39.01,73.37,7.98,86.98,13.30,91.72,18.09,94.67,22.16,94.67,26.42,0.1031
226,coatnet_default_adam_chained_lr_5e-05_partial_...,coatnet,2,95.86,34.22,69.23,4.61,81.66,10.28,89.35,14.18,93.49,18.09,93.49,21.63,0.0378
153,moat_default_window_adam_CosineAnnealingLR_lr_...,moat,2_window,86.98,37.06,65.09,7.27,77.51,11.70,81.07,15.78,84.02,20.39,84.02,24.11,0.0300
323,moat_bigger2_window8_adam_chained_lr_0.0001_wd...,moat,4_window8,92.90,30.85,63.91,6.74,77.51,11.35,84.02,15.07,88.17,17.20,88.17,19.86,0.1942
325,moat_bigger2_window16_adam_CosineAnnealingLR_l...,moat,4_window16,92.90,35.64,59.17,6.38,77.51,12.59,84.02,16.49,89.94,19.15,89.94,22.70,0.1956


In [ ]:
custom_noisy_alphanum = filter_for_dataset(runs, "custom_noisy", False)
custom_noisy_alphanum.to_csv("../results_summary/custom_noisy_alphanum.csv", index=False)
custom_noisy_alphanum

,Name,Model,Architecture,test_acc10_custom_mac,test_acc10_custom_noisy,test_acc1_custom_dishwasher,test_acc1_custom_mac,test_acc1_custom_noisy,test_acc1_custom_open_window,test_acc1_custom_washing_machine,test_acc2_custom_mac,test_acc2_custom_noisy,test_acc3_custom_mac,test_acc3_custom_noisy,test_acc4_custom_mac,test_acc4_custom_noisy,test_acc5_custom_mac,test_acc5_custom_noisy,test_loss_custom_noisy
263,coatnet_bigger2_adam_chained_lr_0.0001_partial...,coatnet,4,92.0,99.07,75.00,63.0,72.14,71.03,70.37,77.0,90.09,86.0,94.12,92.0,95.36,92.0,96.90,0.0179
375,moat_bigger_window_adam_chained_lr_0.0001_wd_0...,moat,3_window,91.0,97.83,76.85,58.0,71.83,70.09,68.52,70.0,85.76,80.0,90.40,85.0,93.19,85.0,95.67,0.0212
175,moat_default_window_adam_CosineAnnealingLR_lr_...,moat,2_window,93.0,97.83,76.85,63.0,70.28,67.29,66.67,76.0,82.97,86.0,89.16,90.0,93.19,90.0,94.74,0.0215
269,coatnet_first_adam_CosineAnnealingLR_lr_0.0001...,coatnet,1,84.0,99.38,69.44,39.0,65.63,60.75,66.67,64.0,82.66,73.0,88.54,78.0,93.50,78.0,95.67,0.0241
273,coatnet_default_adam_CosineAnnealingLR_lr_0.00...,coatnet,2,80.0,97.52,65.74,41.0,62.85,60.75,62.04,53.0,78.64,66.0,87.31,76.0,90.09,76.0,92.88,0.0266
369,moat_bigger_adam_CosineAnnealingLR_lr_0.0001_w...,moat,3,71.0,96.90,62.04,23.0,56.97,58.88,50.00,44.0,72.76,55.0,80.80,66.0,87.00,66.0,89.16,0.0358
386,moat_bigger2_window16_adam_CosineAnnealingLR_l...,moat,4_window16,61.0,94.12,49.07,24.0,45.20,44.86,41.67,39.0,63.16,49.0,70.90,57.0,77.09,57.0,83.59,0.1662
122,moat_default_adam_CosineAnnealingLR_lr_0.0001_...,moat,2,57.0,91.64,41.67,23.0,39.63,41.12,36.11,35.0,56.97,50.0,66.87,54.0,72.76,54.0,79.57,0.0395
178,moat_smaller_window_adam_CosineAnnealingLR_lr_...,moat,1_window,61.0,92.57,37.04,20.0,36.84,42.99,30.56,32.0,56.04,44.0,65.63,53.0,74.92,53.0,81.73,0.0231
383,moat_bigger2_window8_adam_chained_lr_0.0001_wd...,moat,4_window8,55.0,92.26,39.81,23.0,35.91,36.45,31.48,39.0,52.63,49.0,62.54,52.0,73.07,52.0,77.71,0.2156


In [ ]:
custom_noisy_all = filter_for_dataset(runs, "custom_noisy", True)
custom_noisy_all.to_csv("../results_summary/custom_noisy_all.csv", index=False)
custom_noisy_all


,Name,Model,Architecture,test_acc10_custom_mac,test_acc10_custom_noisy,test_acc1_custom_dishwasher,test_acc1_custom_mac,test_acc1_custom_noisy,test_acc1_custom_open_window,test_acc1_custom_washing_machine,test_acc2_custom_mac,test_acc2_custom_noisy,test_acc3_custom_mac,test_acc3_custom_noisy,test_acc4_custom_mac,test_acc4_custom_noisy,test_acc5_custom_mac,test_acc5_custom_noisy,test_loss_custom_noisy
244,coatnet_bigger2_adam_CosineAnnealingLR_lr_0.00...,coatnet,4,96.45,96.99,84.57,71.01,78.72,73.80,77.78,86.98,89.18,91.72,92.20,94.67,93.09,94.67,95.04,0.0085
365,moat_bigger2_window16_adam_CosineAnnealingLR_l...,moat,4_window16,91.72,96.99,85.11,66.27,78.19,68.45,80.95,81.07,89.36,88.17,92.91,90.53,94.68,90.53,95.39,0.0097
349,moat_bigger_adam_CosineAnnealingLR_lr_0.0001_w...,moat,3,93.49,97.52,82.98,67.46,78.01,73.26,77.78,84.02,88.48,89.35,92.91,91.12,94.33,91.12,95.57,0.0095
248,coatnet_bigger_adam_CosineAnnealingLR_lr_0.000...,coatnet,3,94.67,96.28,84.57,72.78,77.30,74.33,73.02,86.39,85.99,89.94,90.43,92.90,91.84,92.90,93.79,0.0096
363,moat_bigger2_window8_adam_chained_lr_0.0001_wd...,moat,4_window8,92.90,96.63,78.72,65.68,76.60,77.01,74.07,79.88,86.88,86.98,90.43,91.72,92.20,91.72,93.79,0.0109
354,moat_bigger_window_adam_CosineAnnealingLR_lr_0...,moat,3_window,85.80,97.16,78.72,57.99,75.35,73.26,74.07,75.74,86.35,81.66,91.13,83.43,93.44,83.43,94.68,0.0111
185,moat_default_window_adam_chained_lr_0.0001_wd_...,moat,2_window,89.35,97.34,78.72,57.40,75.18,74.33,72.49,75.74,86.35,82.25,89.36,86.39,92.02,86.39,93.97,0.0116
256,coatnet_default_adam_CosineAnnealingLR_lr_0.00...,coatnet,2,82.25,96.28,73.40,45.56,67.91,63.64,66.67,66.27,82.80,76.33,87.06,79.88,90.07,79.88,92.02,0.0143
255,coatnet_first_adam_chained_lr_5e-05_partial_wd...,coatnet,1,82.84,96.10,70.74,38.46,64.36,62.03,60.32,57.40,78.55,68.64,84.93,78.70,88.83,78.70,90.60,0.0165
117,moat_default_adam_CosineAnnealingLR_lr_0.0001_...,moat,2,65.68,92.38,56.38,36.69,57.98,55.61,61.90,44.38,72.16,53.25,77.84,64.50,82.09,64.50,84.75,0.0105


In [ ]:
custom_alphanum = filter_for_dataset(runs, "custom", False)
custom_alphanum.to_csv("../results_summary/custom_alphanum.csv", index=False)
custom_alphanum

,Name,Model,Architecture,test_acc10_custom_mac,test_acc10_custom_noisy,test_acc1_custom,test_acc1_custom_dishwasher,test_acc1_custom_mac,test_acc1_custom_noisy,test_acc1_custom_open_window,test_acc1_custom_washing_machine,test_acc2_custom_mac,test_acc2_custom_noisy,test_acc3_custom_mac,test_acc3_custom_noisy,test_acc4_custom_mac,test_acc4_custom_noisy,test_acc5_custom_mac,test_acc5_custom_noisy,test_loss_custom
412,moat_bigger_adam_chained_lr_0.0001_wd_0.05_spe...,moat,3,98.0,98.76,81.09,82.41,95.0,76.78,69.16,78.70,98.0,87.00,98.0,91.64,98.0,93.50,98.0,96.28,0.0062
293,coatnet_bigger2_adam_CosineAnnealingLR_lr_0.00...,coatnet,4,100.0,98.45,80.38,83.33,95.0,75.85,71.96,72.22,100.0,91.33,100.0,94.74,100.0,96.28,100.0,96.28,0.0054
416,moat_bigger_window_adam_chained_lr_0.0001_wd_0...,moat,3_window,100.0,98.45,78.49,82.41,89.0,75.23,68.22,75.00,98.0,89.16,99.0,91.95,100.0,95.36,100.0,96.28,0.0064
297,coatnet_bigger_adam_CosineAnnealingLR_lr_0.000...,coatnet,3,100.0,98.76,74.70,71.30,92.0,69.35,65.42,71.30,99.0,83.90,99.0,90.40,100.0,93.81,100.0,95.67,0.0075
200,moat_default_window_adam_chained_lr_0.0001_wd_...,moat,2_window,98.0,96.90,72.81,75.00,85.0,69.04,62.62,69.44,94.0,84.21,96.0,89.47,98.0,93.19,98.0,94.74,0.0088
305,coatnet_default_adam_CosineAnnealingLR_lr_0.00...,coatnet,2,100.0,98.14,72.34,71.30,88.0,67.49,61.68,69.44,96.0,82.35,97.0,88.85,99.0,92.57,99.0,94.43,0.0084
301,coatnet_first_adam_CosineAnnealingLR_lr_0.0001...,coatnet,1,100.0,97.21,70.45,70.37,84.0,66.25,63.55,64.81,97.0,82.66,99.0,88.54,99.0,93.19,99.0,94.43,0.0084
418,moat_bigger2_adam_CosineAnnealingLR_lr_0.0001_...,moat,4,99.0,98.14,69.27,71.30,84.0,64.71,62.62,60.19,96.0,80.19,99.0,87.31,99.0,91.64,99.0,94.43,0.0138
426,moat_bigger2_window16_adam_CosineAnnealingLR_l...,moat,4_window16,99.0,97.21,67.61,70.37,86.0,61.92,57.01,58.33,94.0,80.80,96.0,86.38,99.0,90.40,99.0,93.19,0.0129
136,moat_default_adam_chained_lr_0.0001_wd_0.05_sp...,moat,2,96.0,95.05,52.96,48.15,66.0,48.92,48.60,50.00,85.0,66.56,91.0,76.47,96.0,81.73,96.0,86.38,0.0150


In [ ]:
custom_all = filter_for_dataset(runs, "custom", True)
custom_all.to_csv("../results_summary/custom_all.csv", index=False)
custom_all


,Name,Model,Architecture,test_acc10_custom_mac,test_acc10_custom_noisy,test_acc1_custom,test_acc1_custom_dishwasher,test_acc1_custom_mac,test_acc1_custom_noisy,test_acc1_custom_open_window,test_acc1_custom_washing_machine,test_acc2_custom_mac,test_acc2_custom_noisy,test_acc3_custom_mac,test_acc3_custom_noisy,test_acc4_custom_mac,test_acc4_custom_noisy,test_acc5_custom_mac,test_acc5_custom_noisy,test_loss_custom
406,moat_bigger2_window16_adam_CosineAnnealingLR_l...,moat,4_window16,100.00,97.52,85.68,87.77,95.86,82.62,76.47,83.60,98.82,90.78,100.00,93.62,100.00,95.04,100.00,95.74,0.0058
390,moat_bigger_adam_CosineAnnealingLR_lr_0.0001_w...,moat,3,100.00,96.63,85.40,87.23,95.86,82.27,78.07,81.48,98.82,90.96,100.00,93.62,100.00,94.50,100.00,95.04,0.0061
394,moat_bigger_window_adam_CosineAnnealingLR_lr_0...,moat,3_window,100.00,97.34,85.27,88.30,94.67,82.45,75.40,83.60,98.22,89.36,98.82,93.44,99.41,95.21,99.41,95.57,0.0062
279,coatnet_bigger2_adam_chained_lr_0.0001_partial...,coatnet,4,100.00,97.87,84.72,87.77,95.27,81.56,74.87,82.01,99.41,90.96,99.41,93.62,100.00,95.74,100.00,96.28,0.0057
282,coatnet_bigger_adam_chained_lr_0.0001_partial_...,coatnet,3,99.41,96.63,84.45,84.04,96.45,80.85,78.07,80.42,99.41,88.12,99.41,91.31,99.41,93.09,99.41,95.39,0.0068
404,moat_bigger2_window8_adam_chained_lr_0.0001_wd...,moat,4_window8,100.00,97.34,84.17,86.70,91.72,81.91,78.61,80.42,98.22,90.60,100.00,94.33,100.00,95.74,100.00,96.28,0.0066
400,moat_bigger2_adam_chained_lr_0.0001_wd_0.05_sp...,moat,4,100.00,97.70,83.77,85.64,92.90,81.03,77.54,79.89,99.41,90.60,100.00,95.04,100.00,96.28,100.00,96.45,0.0066
188,moat_default_window_adam_chained_lr_0.0001_wd_...,moat,2_window,100.00,97.34,82.54,83.51,91.72,79.79,75.40,80.42,95.86,89.36,98.82,93.09,99.41,94.50,99.41,95.57,0.0068
119,moat_smaller_window_adam_chained_lr_0.0001_wd_...,moat,1_window,98.22,95.57,80.49,84.04,89.35,77.84,72.73,76.72,96.45,88.83,98.22,91.13,98.22,92.55,98.22,93.44,0.0046
287,coatnet_first_adam_chained_lr_0.0001_partial_w...,coatnet,1,98.82,96.28,75.44,73.94,91.12,70.74,68.98,69.31,94.08,83.51,98.22,87.77,98.82,90.07,98.82,91.67,0.0105


## Evaluation of Best Models

In [1]:
import os
import sys

sys.path.append(os.path.abspath("./../src"))
from time import sleep

from utils.evaluation import (
    process_sentences_with_model,
    enhance_predictions_with_llm,
    save_results,
)

from dotenv import load_dotenv

load_dotenv()

/Users/juliaprzybytniowska/miniconda3/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'Artificial Intelligence has been part of our world for 75 years now.': ['caps', 'a', 'caps', 'r', 't', 'i', 'f', 'i', 'c', 'i', 'a', 'l', 'space', 'lshift', 'i', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e', 'space', 'h', 'a', 's', 'space', 'b', 'e', 'e', 'n', 'space', 'p', 'a', 'r', 't', 'space', 'o', 'f', 'space', 'o', 'u', 'r', 'space', 'w', 'o', 'r', 'l', 'd', 'space', 'f', 'o', 'r', 'space', '7', '5', 'space', 'y', 'e', 'a', 'r', 's', 'space', 'n', 'o', 'w', 'dot'], 'In order to succeed, we must first believe that we can.': ['lshift', 'i', 'n', 'space', 'o', 'r', 'd', 'e', 'r', 'space', 't', 'o', 'space', 's', 'u', 'c', 'c', 'e', 'e', 'd', 'comma', 'space', 'w', 'e', 'space', 'm', 'u', 's', 't', 'space', 'f', 'i', 'r', 's', 't', 'space', 'b', 'e', 'l', 'i', 'e', 'v', 'e', 'space', 't', 'h', 'a', 't', 'space', 'w', 'e', 'space', 'c', 'a', 'n', 'dot'], 'Life is a journey, not a destination.': ['rshift', 'l', 'i', 'f', 'e', 'space', 'i', 's', 'space', 'a', 'space', 'j', 'o

True

In [2]:
ROOT = "/Users/juliaprzybytniowska/Documents/Master Degree/Master Thesis/KeystrokeDetection/data/final/test/"
MODEL_ROOT = "/Users/juliaprzybytniowska/Desktop/interim_coatnet_segmented/all_dataset"

In [3]:
os.listdir('.')

['evaluation.ipynb', 'audio_preprocessing.ipynb', 'tested_architectures.ipynb']

### Alphanumeric

In [ ]:
alnum_best_models = [
    'coatnet_bigger_adam_CosineAnnealingLR_lr_0.0001_partial_wd_0.05_special_keys_False_all_128_in_lr_True',
    'coatnet_bigger2_adam_CosineAnnealingLR_lr_0.0001_partial_wd_0.05_special_keys_False_all_128_in_lr_True',
    'moat_default_window_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_False_all_64_in_lr_True',
    'moat_bigger2_window8_adam_CosineAnnealingLR_lr_5e-05_wd_0.05_special_keys_False_all_128_in_lr_True',
    'moat_smaller_window_adam_CosineAnnealingLR_lr_5e-05_wd_0.05_special_keys_False_all_128_in_lr_True',
    # 'moat_bigger_adam_CosineAnnealingLR_lr_3e-05_wd_0.05_special_keys_False_all_64_in_lr_True',
    'moat_bigger2_window16_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_False_all_128_in_lr_True',
    'coatnet_first_adam_CosineAnnealingLR_lr_0.0001_partial_wd_0.05_special_keys_False_all_128_in_lr_True',
    'coatnet_default_adam_chained_lr_5e-05_partial_wd_0.05_special_keys_False_all_128_in_lr_True',
    # 'moat_default_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_False_all_128_in_lr_True',
    'moat_bigger2_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_False_all_128_in_lr_True',
    'moat_bigger_window_adam_CosineAnnealingLR_lr_3e-05_wd_0.05_special_keys_False_all_64_in_lr_True',
    # 'moat_smaller_adam_chained_lr_0.0001_wd_0.05_special_keys_False_all_128_in_lr_True',
    'swintransformer_big_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_False_all_128_in_lr_True',
    'swintransformer_default_adam_chained_lr_0.0001_wd_0.05_special_keys_False_all_128_in_lr_True',
    'swintransformer_small_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_False_all_128_in_lr_True',
    'swintransformer_bigger_adam_chained_lr_0.0001_wd_0.05_special_keys_False_all_128_in_lr_True'
]


In [ ]:
for model_name in alnum_best_models:
    print(f"Processing model: {model_name}")
    name = model_name.split("_")[0]
    results = process_sentences_with_model(ROOT, MODEL_ROOT, model_name)
    enhanced_results = enhance_predictions_with_llm(
        results, model_name="gemini", temperature=0.0
    )
    enhanced_results = enhance_predictions_with_llm(
        enhanced_results, model_name="gpt-oss", temperature=0.0
    )
    save_results(enhanced_results, "../results_summary/evaluation/", filename=model_name + ".csv")
    # sleep()

### All keys 

In [ ]:
special_best_models = [
    'coatnet_bigger_adam_CosineAnnealingLR_lr_0.0001_partial_wd_0.05_special_keys_True_all_128_in_lr_True_unmapped',
    'coatnet_bigger2_adam_chained_lr_0.0001_partial_wd_0.05_special_keys_True_all_256_in_lr_True_unmapped',
    # 'moat_bigger_adam_CosineAnnealingLR_lr_5e-05_wd_0.05_special_keys_True_all_128_in_lr_True_unmapped',
    # 'moat_smaller_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_True_all_256_in_lr_True_unmapped',
    'moat_bigger_window_adam_chained_lr_5e-05_wd_0.05_special_keys_True_all_64_in_lr_True_unmapped',
    'moat_smaller_window_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_True_all_256_in_lr_True_unmapped',
    'moat_default_window_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_True_all_128_in_lr_True_unmapped',
    'coatnet_first_adam_chained_lr_0.0001_partial_wd_0.05_special_keys_True_all_256_in_lr_True_unmapped',
    'coatnet_default_adam_chained_lr_5e-05_partial_wd_0.05_special_keys_True_all_128_in_lr_True_unmapped',
    'swintransformer_big_adam_chained_lr_0.0001_wd_0.05_special_keys_True_all_256_in_lr_True_unmapped',
    'swintransformer_default_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_True_all_256_in_lr_True_unmapped',
    'swintransformer_bigger_adam_CosineAnnealingLR_lr_5e-05_wd_0.05_special_keys_True_all_256_in_lr_True_unmapped',
    'swintransformer_small_adam_CosineAnnealingLR_lr_0.0001_wd_0.05_special_keys_True_all_256_in_lr_True_unmapped'
]

In [ ]:
for model_name in special_best_models:
    print(f"Processing model: {model_name}")
    name = model_name.split("_")[0]
    results = process_sentences_with_model(ROOT, MODEL_ROOT, model_name)
    print("Running Gemini enhancement...")
    enhanced_results = enhance_predictions_with_llm(
        results, model_name="gemini", temperature=0.0
    )
    print("Running GPT-OSS enhancement...")
    enhanced_results = enhance_predictions_with_llm(
        enhanced_results, model_name="gpt-oss", temperature=0.0
    )
    save_results(enhanced_results, "../results_summary/evaluation/", filename=model_name + ".csv")
    sleep(30)